# Scraping news
Leggo il file json contenente i link per ogni data e apro il primo link della prima data.
Vado a fare scraping ed ottengo il testo.

In [319]:
import json
import pandas as pd

file_links = open('keyDate_valueLinks.json')  
links = json.load(file_links)
  
file_links.close()

In [320]:
import requests
from bs4 import BeautifulSoup

headers = {'User-Agent': 
                'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

In [321]:
results = pd.read_csv('allMatchesWithResult.csv')

## Ottenimento previsioni

In [317]:
"""
I nomi delle squadre che vengono letti da football predictions non sono normalizzati quindi bisogna cambiare il nome.

Prendo i nomi delle squadre salvate sul file dei risultati e prendo il nome della squadra prese da football predictions. Se trovo la corrispondenza, converto il nome della squadra, altrimenti lascio il nome attuale.

Ho visto che ad esempio invece di scrivere Cagliari, è stato scritto Calgiari. Controllo la percentuale di similarità tra le due stringhe e se varia di una lettera, lo score sarà alto (maggiore di 0.8 --> 0.857 per l'esattezza)
"""
from difflib import SequenceMatcher

originalNotation = results['h_team'].unique()

def check_name(new_name):
    name = new_name
    for team_name in originalNotation:
        if (team_name in new_name) or (new_name in team_name) or (SequenceMatcher(None, new_name, team_name).ratio() > 0.8): 
            return team_name
    return new_name

get_prediction è il metodo che permette di ottenere 3 dati: 
- il nome della squadra di casa
- il nome della squadra di trasferta
- la predizione (descrizione)

In [318]:
def get_prediction(prediction_link):
    pageTree = requests.get(prediction_link)
    soup = BeautifulSoup(pageTree.text, features="lxml")

    predtxt = soup.find("span", {"class": "predtxt-mob"})
    text = []

    paragraph = predtxt.find_all('p')
    for p in paragraph:
        text.append(p.text)

    paragraph_team = soup.find_all("p", {"class": "teamnaam"})
    return paragraph_team[0].contents[0], paragraph_team[1].contents[0], text

Setto le informazioni trovate al link nel dataframe

In [322]:
from tqdm import tqdm
df = pd.DataFrame()
count = 0
for key_date, links_of_predictions in tqdm(links.items()):
    for prediction_link in links_of_predictions:
        home, away, text = get_prediction(prediction_link)
        df.at[count, 'date'] = key_date        
        df.at[count, 'h_team'] = check_name(home)
        df.at[count, 'a_team'] = check_name(away)
        
        text = '\n\n'.join(text)
        df.at[count, 'description'] = text
        
        count += 1

100%|██████████| 119/119 [06:59<00:00,  3.53s/it]


Dato che il procedimento di scaricamento delle descrizioni è molto lungo, tutti i dati sono salvati nel seguente csv. Il problema è che i dati devono essere elaborati, quindi guardare le sezioni successive

In [392]:
df.to_csv("all_correspondeces.csv")

### Recuperi partite

Quello descritto sopra non è l'unico problema, in quanto ci siano partite che sono state rimandate. Ovviamente i pronostici vengono fatti su due date diverse:
- sulla data originale della partita (che poi viene rimandata)
- sulla data rimandata
Per questo devo cancellare le righe relative alle partite che non si sono più svolte in quella data e tenere solo i recuperi

In [380]:
recoveries = df.copy()

for k, match in results.iterrows():
    h_team, a_team = match.h_team, match.a_team
    recovery = recoveries[(recoveries['h_team']==h_team) & (recoveries['a_team']==a_team)]
    if len(recovery) > 1:
        rec = (recoveries[(recoveries['h_team']==h_team) & (recoveries['a_team']==a_team)])[1:]
        recoveries.drop((recoveries[(recoveries['h_team']==h_team) & (recoveries['a_team']==a_team)])[:1].index, inplace=True)


range_index = [i for i in range(len(recoveries))]

Devo fare il reset dell'index. Questo perché quando cerco di inserire un nuovo elemento nel dataframe, avendo eliminato in precedenza le partite "fasulle" (perché poi ci sono i recuperi) nel dataframe, quando gli passo l'indice questo non è relativo alla posizione, ma al vecchio indice e quindi se inserisco un valore con quel valore, potrebbe sovrascrivere quel record.
Ad esempio, abbiamo 379 record. 
C'è un record con indice 379, se io cercassi di fare df.at(379, ['example']) = 0, all'indice 379 (non alla posizione) viene settata a 0 la colonna 'example'. 


In [383]:
recoveries['index'] = range_index 
recoveries = recoveries.set_index('index') 

### Partite non trovate

Ci potrebbero essere partite che non sono state trovate, quindi confronto i match che ci sono nel dataframe dei risultati e in quello scaricato da football predictions.
C'è quindi un altro problema: i link potrebbero essere errati, quindi con lo scraping non vengono prelevati. 

Bisogna cercare nel giorno della partita e cercare nella relativa pagina l'href che contiene la partita.

In [387]:
count = len(recoveries)
for k, res in results.iterrows():
    if len(recoveries[(recoveries['h_team']==res.h_team) & (recoveries['a_team']==res.a_team)]) < 1:
        print(res.date, res.h_team, res.a_team) #stampo quali sono i match che non sono stati trovati
        
        converted_data = pd.to_datetime(res.date, format='%Y-%m-%d', errors='coerce').date()
        new_date = converted_data.strftime('%d-%m-%Y')

        pageTree = requests.get('https://footballpredictions.com/footballpredictions/?date='+new_date)
        soup = BeautifulSoup(pageTree.text, features="lxml")
        links = soup.find_all('a', href=True)
        links = [link['href'] for link in links if (res.h_team.lower() in link['href']) or (res.a_team.lower() in link['href'])]
        cleaned_links = list(dict.fromkeys(links))
        if len(cleaned_links) > 0: 
            home, away, text = get_prediction(cleaned_links[0])
            
            recoveries.at[count, 'date'] = res.date
            recoveries.at[count, 'h_team'] = check_name(home)
            recoveries.at[count, 'a_team'] = check_name(away)
            
            text = '\n\n'.join(text)
            recoveries.at[count, 'description'] = text

            count += 1 #questo contatore serve per inserire in una determinata posizione
        

2022-03-06 Genoa Empoli


Ho visto che alcune date non combaciano, in quanto su football predictions alcune date sono sballate (non combaciano al giorno stesso effettivo della partita, ma al giorno precedente) quindi per fare il match tra i due dataset (questo con le descrizioni delle predizioni e quello del match con la data corretta) devo considerare squadra home e away.

In [389]:
recoveries[['date', 'h_team', 'a_team']]

,date,h_team,a_team
index,,,
0,2021-08-21,Inter,Genoa
1,2021-08-21,Hellas Verona,Sassuolo
2,2021-08-21,Empoli,Lazio
3,2021-08-21,Torino,Atalanta
4,2021-08-22,Bologna,Salernitana
...,...,...,...
375,2022-05-22,Inter,Sampdoria
376,2022-05-22,Sassuolo,Milan
377,2022-05-22,Salernitana,Udinese


In [390]:
dataset = pd.merge(results[['date', 'h_team', 'a_team', 'result']], recoveries[['h_team','a_team', 'description']], on=['h_team', 'a_team'])
dataset.to_csv('description_predictions.csv')